In [1]:
import fleet as flt


In [4]:
fleet = flt.AsyncFleet(api_key="sk_altYhLjKdJ2ixgfX2zefJFKAOpVGIU3rjSDZ6n3NhKw")

environments = await fleet.list_envs()
print("Environments:", len(environments))

# Create a new instance
instance = await fleet.make(flt.InstanceRequest(env_key="hubspot", version="v1.2.7"))
print("New Instance:", instance.instance_id)

environment = await fleet.environment(instance.env_key)
print("Environment Default Version:", environment.default_version)

Environments: 3
[{'name': 'seed', 'type': 'sqlite', 'mode': 'ro', 'label': 'Seed database'}, {'name': 'current', 'type': 'sqlite', 'mode': 'rw', 'label': 'Current database'}, {'name': 'cdp', 'type': 'cdp', 'mode': 'rw', 'label': 'CDP'}]
New Instance: afa358cc
Environment Default Version: v1.2.6


In [5]:
response = await instance.env.reset(flt.ResetRequest(seed=42))
print("Reset response:", response)

print(await instance.env.resources())

sqlite = instance.env.db("current")
print("SQLite:", await sqlite.describe())

Reset response: success=True message='Database reset successfully'
[<fleet.resources.sqlite.AsyncSQLiteResource object at 0x113932570>, <fleet.resources.browser.AsyncBrowserResource object at 0x1156e5bb0>]
SQLite: success=True resource_name='current' tables=[TableSchema(name='activities', sql="CREATE TABLE activities (\n            id INTEGER PRIMARY KEY AUTOINCREMENT,\n            user_id INTEGER NOT NULL, -- User who performed the activity\n            owner_id INTEGER, -- User who owns the activity\n            activity_type TEXT NOT NULL, -- 'email', 'call', 'meeting', 'note', 'stage_change', 'task'\n            subject TEXT,\n            body TEXT,\n            metadata TEXT, -- JSON for additional data like duration, outcome, etc.\n            activity_date DATETIME NOT NULL,\n            createdAt DATETIME DEFAULT CURRENT_TIMESTAMP,\n            FOREIGN KEY (user_id) REFERENCES users(id),\n            FOREIGN KEY (owner_id) REFERENCES users(id)\n        )", columns=[{'cid': 0, '

In [6]:
print("Query:", await sqlite.query("SELECT * FROM users"))

Query: success=True columns=['id', 'email', 'display_name', 'me'] rows=[[1, 'sarah.kim@company.com', 'Sarah Kim', 1], [2, 'john.smith@company.com', 'John Smith', 0], [3, 'emily.chen@company.com', 'Emily Chen', 0], [4, 'michael.brown@company.com', 'Michael Brown', 0], [5, 'lisa.wang@company.com', 'Lisa Wang', 0], [6, 'david.johnson@company.com', 'David Johnson', 0], [7, 'jennifer.davis@company.com', 'Jennifer Davis', 0], [8, 'robert.wilson@company.com', 'Robert Wilson', 0], [9, 'maria.garcia@company.com', 'Maria Garcia', 0], [10, 'james.anderson@company.com', 'James Anderson', 0], [11, 'support.team@company.com', 'Support Team', 0], [12, 'mike.johnson@company.com', 'Mike Johnson', 0], [13, 'edward.jackson@nimbusai.com', 'Edward Jackson', 0], [14, 'sarah.kim@nimbus.ai', 'Sarah Kim', 1], [15, 'betty.alexander@nimbusai.com', 'Betty Alexander', 0], [16, 'raj_patel@nimbusai.ai', 'Raj Patel', 0], [17, 'melissa.carter@nimbusai.com', 'Melissa Carter', 0], [18, 'marcus_johnson@nimbusai.ai', 'Mar